In [28]:
import pandas as pd
from pandas.io.json import json_normalize
import json
import pyodbc
import os
import numpy as np
import time

In [29]:
big_df = pd.read_csv('tmdb_5000_movies.csv')

In [3]:
big_df

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [30]:
big_df.drop(columns=['original_title', 'status','homepage','tagline','vote_count','id'],inplace=True)
big_df['popularity'] = big_df['popularity'].apply(lambda x: round(x, 2))
big_df['vote_average'] = big_df['vote_average'].apply(lambda x: round(x, 1))
big_df['runtime'] = big_df['runtime'].fillna(0).astype(int)
big_df.insert(0,'Id',range(1,1+len(big_df)))
big_df


,Id,budget,genres,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,title,vote_average
0,1,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,"In the 22nd century, a paraplegic Marine is di...",150.44,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Avatar,7.2
1,2,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,"Captain Barbossa, long believed to be dead, ha...",139.08,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Pirates of the Caribbean: At World's End,6.9
2,3,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,A cryptic message from Bond’s past sends him o...,107.38,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Spectre,6.3
3,4,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,Following the death of District Attorney Harve...,112.31,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",The Dark Knight Rises,7.6
4,5,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,"John Carter is a war-weary, former military ca...",43.93,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",John Carter,6.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4799,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi just wants to play his guitar and ...,14.27,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",El Mariachi,6.6
4799,4800,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",[],en,A newlywed couple's honeymoon is upended by th...,0.64,[],[],2011-12-26,0,85,[],Newlyweds,5.9
4800,4801,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"""Signed, Sealed, Delivered"" introduces a dedic...",1.44,"[{""name"": ""Front Street Pictures"", ""id"": 3958}...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2013-10-13,0,120,"[{""iso_639_1"": ""en"", ""name"": ""English""}]","Signed, Sealed, Delivered",7.0
4801,4802,0,[],[],en,When ambitious New York attorney Sam is sent t...,0.86,[],"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-05-03,0,98,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Shanghai Calling,5.7


In [7]:
def json_to_df(json_col):
    dfs = []
    df = big_df[['Id',json_col]].copy()
    

    def parse_json(row):
        json_df = pd.read_json(row[json_col])
        dfs.append(json_df.assign(Film_Id = row['Id']))

    df.apply(parse_json, axis=1)
    film_subfact = pd.concat(dfs, ignore_index = True)
    
#     Move the FilmId col to the first position
    col = film_subfact.pop('Film_Id')
    film_subfact.insert(0, 'Film_Id', col)
    
# Change the columns' name
    film_subfact.drop(film_subfact.columns[1],axis=1,inplace=True)
    film_subfact.columns = ['Film_Id',json_col.capitalize()]

# Capitalize the values in the Dim column to make sre we remove all duplicates (in Python COMEDY vs Comedy is different but not in SQL)
# Not do this for Film table because films can have the same name
    film_subfact[json_col.capitalize()] = film_subfact[json_col.capitalize()].str.capitalize()
    
# Create film dim fact (granularity level)
    col_name = json_col.capitalize() + '_Id'
    film_dim = film_subfact.drop_duplicates(subset=[json_col.capitalize()],ignore_index=True)
    film_dim.drop(columns=['Film_Id'],inplace=True)
    film_dim.insert(0,col_name,range(1,1+len(film_dim)))
#     film_dim.columns = []
    
# Merge the tables to map back the Id into the subfact table
    merged_subfact = film_subfact.merge(film_dim,how='left',on=json_col.capitalize())

# Remove the ID column in film dim (will add by auto increment in database later)
    film_dim.drop(columns=[col_name],inplace=True)
    
# Remove the fact column from film fact table
    merged_subfact.drop(columns=[json_col.capitalize()],inplace=True)
    
#     merged_subfact.drop(columns=[json_col.capitalize()+'_x'],inplace=True)
    return merged_subfact,film_dim






In [8]:
film_genres,genres = json_to_df('genres')
film_keywords,keywords = json_to_df('keywords')
film_production_countries,production_countries = json_to_df('production_countries')
film_spoken_languages,spoken_languages = json_to_df('spoken_languages')

C:\Users\nhatt\AppData\Local\Temp\ipykernel_23324\861939512.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  film_dim.drop(columns=['Film_Id'],inplace=True)
C:\Users\nhatt\AppData\Local\Temp\ipykernel_23324\861939512.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  film_dim.drop(columns=[col_name],inplace=True)
C:\Users\nhatt\AppData\Local\Temp\ipykernel_23324\861939512.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  film_

In [6]:
# Production countries need a separate file because the order of id and name are reverse
dfs_pc = []
df_pc = big_df[['Id','production_companies']].copy()
    

def parse_json(row):
    json_df = pd.read_json(row['production_companies'])
    dfs_pc.append(json_df.assign(Film_Id = row['Id']))

df_pc.apply(parse_json, axis=1)
film_subfact_pc = pd.concat(dfs_pc, ignore_index = True)
    
#     Move the FilmId col to the first position
col = film_subfact_pc.pop('Film_Id')
film_subfact_pc.insert(0, 'Film_Id', col)
    
# Change the columns' name
film_subfact_pc.drop(film_subfact_pc.columns[2],axis=1,inplace=True)
film_subfact_pc.columns = ['Film_Id','Production_companies']

# Capitalize the values in the Dim column to make sre we remove all duplicates (in Python COMEDY vs Comedy is different but not in SQL)
# Not do this for Film table because films can have the same name
film_subfact_pc['Production_companies'] = film_subfact_pc['Production_companies'].str.capitalize()
    

# Create film dim fact (granularity level)
col_name = 'Production_companies_Id'
production_companies = film_subfact_pc.drop_duplicates(subset=['Production_companies'],ignore_index=True)
production_companies.drop(columns=['Film_Id'],inplace=True)
production_companies.insert(0,col_name,range(1,1+len(production_companies)))
#     film_dim.columns = []


    
# Merge the tables to map back the Id into the subfact table
film_production_companies = film_subfact_pc.merge(production_companies,how='left',on='Production_companies')

# Remove the ID column in film dim (will add by auto increment in database later)
production_companies.drop(columns=[col_name],inplace=True)

# Remove the production companies from fact table
film_production_companies.drop(columns=['Production_companies'],inplace=True)


C:\Users\nhatt\AppData\Local\Temp\ipykernel_10416\1901235266.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  production_companies.drop(columns=['Film_Id'],inplace=True)
C:\Users\nhatt\AppData\Local\Temp\ipykernel_10416\1901235266.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  production_companies.drop(columns=[col_name],inplace=True)


In [31]:
film = big_df.drop(columns=['genres', 'keywords','production_companies','production_countries','spoken_languages'])
film = film[['title','original_language','release_date','runtime','revenue', 'budget','vote_average','popularity','overview']]
film['overview'] = film['overview'].replace({np.nan:None})
film['release_date'] = pd.to_datetime(film['release_date'], format="%Y-%m-%d")
film['release_date'] = film['release_date'].astype(object).where(film['release_date'].notnull(), None)


In [ ]:
server = 'Who\SQLEXPRESS'
database = 'tmdbMovies'
username = 'abc'
password = '123'
conn_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"

# Create the connection
try:
    conn = pyodbc.connect(conn_string)
except Exception as e:
    print(e)
    print('task is terminated')
    sys.exit()

In [33]:
def insert(df,df_name):
    placeholders = ', '.join(['?' for _ in range(len(df.columns))])
    insert_statement = f"""
    INSERT INTO {df_name}  
    VALUES ({placeholders})
"""

    try:
        cursor = conn.cursor()
        cursor.executemany(insert_statement, list(df.itertuples(index=False, name=None)))
    except Exception as e:
        cursor.rollback()
        print('Error occurred while inserting into', df_name, ':', str(e))
        print('Transaction rolled back for', df_name)
    else:
        print('Records inserted successfully')
        cursor.commit()
        cursor.close()
        time.sleep(30)

In [53]:
insert(film,'film')
insert(film_genres,'film_genres')
insert(genres,'genres')
insert(film_keywords,'film_keywords')
insert(keywords,'keywords')
insert(film_production_companies,'film_production_companies')
insert(production_companies,'production_companies')
insert(film_production_countries,'film_production_countries')
insert(production_countries,'production_countries')
insert(spoken_languages,'spoken_languages')
insert(film_spoken_languages,'film_spoken_languages')

Records inserted successfully
Records inserted successfully
Records inserted successfully
Records inserted successfully
Error occurred while inserting into film_production_companies : ('21S01', '[21S01] [Microsoft][ODBC SQL Server Driver][SQL Server]Column name or number of supplied values does not match table definition. (213) (SQLExecDirectW); [21S01] [Microsoft][ODBC SQL Server Driver][SQL Server]Statement(s) could not be prepared. (8180)')
Transaction rolled back for film_production_companies
Records inserted successfully
Records inserted successfully
Records inserted successfully


In [55]:
insert(film_production_companies,'film_production_companies')

Records inserted successfully


In [ ]:
valid_titles = film['overview'].dropna().astype(str)

# Check if there are any valid titles
if not valid_titles.empty:
    # Find the longest title
    longest_title = max(valid_titles, key=len)
    num_characters = len(longest_title)
    print("Longest file title:", longest_title)
    print("Number of characters:", num_characters)
else:
    print("No valid file titles found in the 'file_title' column.")

In [ ]:
film.to_csv('film.csv',index=False)

Records inserted successfully
